## Quadtree

**Quadtree** *(или квадродерево)* — дерево, каждая внутренняя вершина которого содержит 4 ребёнка. 

Каждой вершине квадродерева соответствует некоторый квадрат. 

Если внутренней вершине $v$ соответствует какой-то квадрат $a$, то её детям этой вершины соответствуют четверти квадрата $a$. 

### Построение Quadtree

Пусть дано множество точек $P$, для которого нужно построить квадродерево. Начнём с некоторого квадрата $\sigma$, содержащего все точки из $P$.

* если $P$ не содержит точек, то корнем квадродерева будет лист, в котором в качестве координат точки будет храниться `None`;
* если $P$ содержит 1 точку то корнем квадродерева будет лист, в котором хранятся координаты единственной точки из $P$;
* иначе корнем дерева будет вершина $v$, которой соответствует квадрат $\sigma$, а её детям будут соответствовать  четверти квадрата $\sigma$. Теперь рекурсивно превращаем каждого ребёнка в квадродерево для множества точек, лежащих в соответствующей четверти.

<img src="src/SimpleQuadtree.svg">

Назовём квадрат **интересным**, если соответствующая ему вершина дерева имеет хотя бы два непустых ребёнка или является корнем.

### Построение Composed Quadtree

Сжатое квадродерево получается сжатием обычного таким образом, чтобы остались только интересные квадраты. 

* Пустые дети неинтересных квадратов удаляются.
* Для каждого интересного квадрата $p$ будем хранить четыре указателя для каждой четверти этого квадрата. 
* Если четверть содержит две или более точки, то указатель ссылается на наибольший интересный квадрат в этой четверти. 
* Если четверть содержит одну точку, то указатель ссылается на эту точку. 
* Eсли четверть не содержит точек, то указатель сделаем нулевым.

<img src="src/SimpleToComposed.svg"/>

**Лемма (о высоте сжатого квадродерева) :**

Сжатое квадродерево для $n$ точек имеет $O(n)$ вершин и глубину $O(n)$.

Докажем по индукции, что в квадродереве для $n$ точек количество интересных квадратов меньше либо равно $n$:

* Для $n = 1$ это очевидно.
* Пусть доказано для квадродерева с $n - 1$ точек. Покажем, что добавление новой точки $x$ изменит количество интересных квадратов не более чем на один.
    * Если $x$ находится в пустой четверти, то просто добавляем $x$ как лист, не изменив число интересных квадратов. 
    * Если же четверть в которую необходимо вставить $x$, уже содержит точку $y$, то мы можем добавить в дерево интересный квадрат, который будет содержать $x$ и $y$ в разных четвертях. 


### Локализация

**Локализация** - поиск наименьшего интересного квадрата, где геометрически находится интересующая точка. 

Начиная с корня, спускаемся вниз по дереву, переходя в те четверти, где должна геометрически лежать искомая точка.

### Вставка

Сначала локализуемся, находим наименьший квадрат, в который надо вставить точку, понимаем, в какую четверть будем вставляться. Рассмотрим эту четверть:

* Если четверть пустая, то просто запоминаем координаты вершины в этом листе.
* Если там есть точка (или интересный квадрат), то заменяем этот лист, на новый интересный квадрат, который в четвертях содержит эти точки (или точку и интересный квадрат).

### Удаление

Сначала локализуемся, находим наименьший интересный квадрат, в котором может лежать искомая точка. Если в одной из четвертей хранится указатель на координаты нашей точки — удаляем её. После удаления точки наш квадрат может перестать быть интересным. При этом, родитель нашего квадрата в любом случае останется интересным. В таком случае нам надо просто заменить наш квадрат на его единственного непустого ребенка.

### Наихудший случай

<img src="src/QuadtreeWorstCase.svg">

## Skip Quadtree

**Skip quadtree** — структура данных, основанная на идее структуры **skip list**, позволяющая хранить множество точек и быстро производить над ним операции, такие как локализация, вставка и удаление.

Мы рассмотрим одну из реализаций — **randomized skip quadtree**.

### Построение Randomized Skip Quadtree

Рассмотрим последовательность вложенных подмножеств $\{S_i\}$ некоторого множества точек $S$. 

* $S_0 = S$ 
* $S_i \subset S_{i-1}$, причем каждый элемент из $S_{i-1}$ входит в $S_{i}$ с вероятностью $p \in \left(0, 1\right)$. 

**Randomized skip quadtree** для множества точек $S$ будет состоять из набора уровней $\{Q_i\}$, где $Q_i$ — сжатое квадродерево над множеством $S_i$.

Также мы должны уметь из вершины с одного уровня перемещаться на соответствующую ей вершину на предыдущем уровне. Это можно делать несколькими способами:
    
* Просто хранить ссылку внутри вершины.
* Каждый уровень будет хранить ассоциативный массив из масок в вершины. Маску для вершины можно однозначно задать по её координатам.

### Локализация

Локализация происходит так же как и в **quadtree**.
Сначала локализуемся в наивысшем уровне, начиная из корня дерева. Из полученной вершины переходим в соответствующую ей вершину на предыдущем уровне. Снова запускаем локализацию, начиная из вершины, полученной предыдущей локализацией. Повторяем процесс пока не локализуемся на нулевом уровне.

### Вставка

1. Локализация. В процессе локализации будем для каждого уровня запоминать вершину, на которой остановилась локализация внутри этого уровня.
2. Вставка точки в нулевой слой, в вершину, полученную локализацией из п.1.
3. С вероятностью $p$, переходим на уровень выше к вершине, которую мы запомнили в п.1, вставляем точку в эту вершину.
4. В случае удачи повторяем п.3, в случае неудачи завершаем операцию.
5. Если мы хотим перейти к слою, которого нет, создаем новый слой, вставляем туда новую точку и завершаем операцию.

### Удаление

Запускаем локализацию. В процессе локализации будем удалять точку на всех уровнях, где она есть. Если на уровне стало ноль точек - удаляем уровень.

**Лемма (о количестве шагов локализации на одном уровне):**

На каждом уровне совершается $O(1)$ шагов локализации для любой точки `x`.

Пусть в на `i`-ом уровне поиск точки `x`, начинающийся с корня, проходит по квадратам $p_0, p_1 \dots p_m$. Пусть случайная величина $j$ — количество шагов локализации внутри $Q_i$, тогда $p_{m - j}$ — последний квадрат из $p_0, p_1 \dots p_m$, являющийся интересным в $Q_{i + 1}$.

Оценим мат.ожидание случайной величины $j$. 

Пусть $T$ - множество встреченных на пути $p_{m - j + 1} \dots p_m$ непустых четвертинок. Чтобы $p_{m - j}$ был последним из $p_0, p_1 \dots p_m$ интересным квадратом в $Q_{i + 1}$ необходимо, чтобы среди $T$ только одна (вероятность этого назовём $\mathrm{Pr}_1$) или ноль (вероятность этого назовём $\mathrm{Pr}_0$) были непустыми в $Q_{i + 1}$. Иначе, если будет хотя бы пара непустых четвертинок, то их наименьший общий предок в дереве будет интересным квадратом и будет находиться глубже $p_{m - j}$.

Таким образом, искомая вероятность не превосходит $\mathrm{Pr}_0 + \mathrm{Pr}_1$.

Пусть $|T| = t$, $q = 1 - p$.

$\mathrm{Pr}_0 = q^t$, потому что $\mathrm{Pr}_0$ означает то, что ни одна точка из $T$ не попала на уровень выше.

$\mathrm{Pr}_1 = t \cdot pq^{t-1}$, потому что $\mathrm{Pr}_1$ означает то, что ровна одна точка из $T$ не попала на уровень выше.

$$\displaystyle
\begin{align}
\mathop{\mathbb{E}}j
& = \sum\limits_{j = 1}^{m} j \cdot \left(\mathrm{Pr}_0 (j) + \mathrm{Pr}_1 (j) \right) \\
& = \sum\limits_{j = 1}^{m} j \left(q^t + t \cdot pq^{t-1}\right) \\
& = \sum\limits_{j = 1}^{m} j ((1-t) \cdot q^t + t \cdot q^{t-1}) \\
\end{align}
$$

В интересной вершине как минимум 2 непустые четверти (в одну из которых мы переходим), всего на пути $j$ интересных вершин, следственно, количество вершин в $T$ - хотя бы $j$. Следственно, количество точек в $T$ - тоже хотя бы $j$.

$$\displaystyle
\begin{align}
\mathop{\mathbb{E}}j
&\leq \sum_{j = 1}^{m} j \left(\left(1-j\right) \cdot q^j + t \cdot q^{j-1}\right) \\
&< \sum_{j = 1}^{\infty} j \left(\left(1-j\right) \cdot q^j + j \cdot q^{j-1}\right) \\
&< \sum_{j = 1}^{\infty} \left(2 j \left(j-1\right) \cdot q^{j-1} - j \left(j-1\right) \cdot q^j\right) \\
&< \sum_{j = 1}^{\infty} 2 j (j-1) \cdot q^{j-1} 
 = 2 \sum_{j = 1}^{\infty} \left(q^{j+1}\right)'' \\
&= 2 \left(\sum_{j = 1}^{\infty} q^{j+1}\right)'' 
 = \left(\frac{2 q^2}{\left(1 - q\right)}\right)'' \\
&= 4p^{-3}
\end{align}
$$

**Лемма (О количестве уровней):** 

Количество уровней — $O(\log n)$

Пусть $h$ — cлучайная величина, обозначающая количество уровней.
Оценим $p(h = k)$. 

$$p(h = k) = p(h < k + 1) - p (h < k)$$

Т.к. вероятность того, что при добавлении точка дойдет до уровня $k$ равняется $p^k$:
$$p(h < k) = \left(1 - p^k\right)^n$$

$$\displaystyle
\begin{align}
p(h = k)
& = \left(1 - p^{k + 1}\right)^n - \left(1 - p^k\right)^n \\
& < 1 - \left(1 - p^k\right)^n \\
& < np^k
\end{align}
$$

Оценим мат.ожидание $h$:

$$ \mathop{\mathbb{E}}h
= \sum_{k = 1}^{\infty}k \cdot p(h = k)
= \left[\sum_{k = 1}^{\log_{1/p}(n)}k \cdot p(h = k)\right]
+ \left[\sum_{k = \log_{1/p}(n) + 1}^{\infty}k \cdot p(h = k)\right]
$$

Первая сумма:
$$\displaystyle
\begin{align}
\sum_{k = 1}^{\log_{1/p}(n)}k \cdot p(h = k) 
& < \sum_{k = 1}^{\log_{1/p}(n)}\log_{1/p}(n) \cdot p(h = k) \\
& = \log_{1/p}(n) \cdot \sum\limits_{k = 1}^{\log_{1/p}(n)} p(h = k) \\
& = O(\log n)
\end{align}
$$

Вторая сумма:
$$\displaystyle
\begin{align}
\sum_{\log_{1/p}(n) + 1}^{\infty}k \cdot p(h = k)
& < \sum_{\log_{1/p}(n) + 1}^{\infty}k \cdot np^k  
  < \sum_{\log_{1/p}(n)}^{\infty}k \cdot np^k 
  = n \cdot \sum_{\log_{1/p}(n)}^{\infty}k p^k \\
& = n p^{\log_{1/p} n} \cdot \sum_{k=0}^{\infty} \left(k + \log_{1/p} n\right) \cdot p^k \\
& = n p^{\log_{1/p} n} \cdot \left(\sum_{k=0}^{\infty} k p^k + \log_{1/p} n \sum_{k=0}^{\infty} p^k\right) \\
& = n p^{\log_{1/p} n} \cdot \left(O(1) + \log_{1/p} n \cdot O(1)\right) \\
& = n \cdot n^{-1} \cdot \log_{1/p} n \cdot O(1) \\
& = O(\log n)
\end{align}
$$

### Оценка памяти

Сжатое квадродерево для $n$ точек занимает $O(n)$ памяти. На нулевом уровне $n$ точек. На следующем уровне $O(p n)$ точек, дальше $O(p^2 n)$ и так далее.
В итоге получаем:
$$\sum\limits_{k=0}^{O(\log n)} O\left(p^k n\right) < \sum\limits_{k=0}^{\infty} O\left(p^k n\right) = O(n) $$